In [260]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_theme()

# optimization
from scipy import optimize

# For quick OLS
#import statsmodels.formula.api as smf

# Import our toolbox
import clogit as clogit
import estimation as est

# Read in data

The dataset, `cars.csv`, contains cleaned and processed data. If you want to make changes, the notebook, `materialize.ipynb`, creates the data from the raw source datsets. 

In [261]:
cars = pd.read_csv('cars.csv')
lbl_vars = pd.read_csv('labels_variables.csv')
lbl_vals = pd.read_csv('labels_values.csv')

# convert from dataframe to dict
lbl_vals = {c: lbl_vals[c].dropna().to_dict() for c in lbl_vals.columns}

In [262]:
lbl_vars.set_index('variable', inplace=True)

## Overview of the dataset

In [263]:
lbl_vars.join(cars.mean(numeric_only=True).apply(lambda x: f'{x: .2f}').to_frame('Mean'))

,label,Mean
variable,,
ye,year (=first dimension of panel),84.50
ma,market (=second dimension of panel),3.00
co,model code (=third dimension of panel),207.50
zcode,alternative model code (predecessors and succe...,177.76
brd,brand code,16.79
type,name of brand and model,NaN
brand,name of brand,NaN
model,name of model,NaN
org,"origin code (demand side, country with which c...",2.72


In [265]:
cars['s'].nunique()

5986

# Set up for analysis

In [266]:
price_var = 'princ'

In [267]:
cars['logp'] = np.log(cars[price_var])

In [268]:
# new variable: price elasticity heterogeneous for home-region 
cars['logp_x_home'] = cars[price_var] * cars['home']

### Dummy variables

For working with matrices, we want to have a column for each dummy variable. 

In [269]:
categorical_var = 'brand' # name of categorical variable
dummies = pd.get_dummies(cars[categorical_var]) # creates a matrix of dummies for each value of dummyvar
x_vars_dummies = list(dummies.columns[1:].values) # omit a reference category, here it is the first (hence columns[1:])

# add dummies to the dataframe 
assert dummies.columns[0] not in cars.columns, f'It looks like you have already added this dummy to the dataframe. Avoid duplicates! '
cars = pd.concat([cars,dummies], axis=1)

In [439]:
# NB! Let's take a look at the reference category
dummies.columns[0] #This is BMW
# Something might be going on 

'BMW'

### `x_vars`: List of regressors to be used 

In [533]:
x_vars = ['logp', 'home', 'cy', 'hp', 'we', 'li'] + ['tax', 'pop', 'inc'] #+ x_vars_dummies 
# If we include dummies for the brand, the 3-D matrix x becomes singular. Why?
# 
print(f'K = {len(x_vars)} variables selected.')

K = 9 variables selected.


In [534]:
cars = cars.dropna(axis=1)

In [535]:
cars['brand'].nunique() #Different car brands

33

In [536]:
cars['co'].nunique() #Different car models

285

In [537]:
M = 30
T = 5
N = M*T
J = 40

In [538]:
K = len(x_vars) #The "household" characteristics
N = cars.ma.nunique() * cars.ye.nunique() #The market-year 'i'
J = 40 #The 40 different cars
x = cars[x_vars].values.reshape((N,J,K))

#cars['y'] = np.resize(np.arange(0, M*T), N*J)
#y = cars['y'].values.reshape((N,J))

#cars['y'] = pd.factorize(cars['co'].values)[0]
#y = pd.factorize(cars['co'].values)[0]
#y = y.reshape((N,J))

cars['y'] = np.resize(np.arange(0,J), N*J)
y = cars['y'].values.reshape((N,J))

#cars['brand']=pd.factorize(cars['brand'])[0]
#y = cars['co'].values.reshape((N,J))
#cars['co']=pd.factorize(cars['co'])[0]

In [539]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39])

In [540]:
y.shape

(150, 40)

In [541]:
y[1,:]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39])

In [517]:
test = cars[(cars['ma']==1) & (cars['ye']==70)]

In [518]:
test

,ye,ma,co,zcode,brd,type,brand,model,org,loc,...,skoda,suzuki,tal/hillman,tal/matra,tal/simca,tal/sunb,talbot,toyota,volvo,y
0,70,1,15,14,2,audi 100/200,audi,100/200,2,4,...,0,0,0,0,0,0,0,0,0,0
1,70,1,26,35,4,citroen 2 CV 6 - 2 CV 4,citroen,2CV6,1,3,...,0,0,0,0,0,0,0,0,0,1
2,70,1,36,36,4,citroen dyane,citroen,dyane,1,3,...,0,0,0,0,0,0,0,0,0,2
3,70,1,64,67,7,fiat 128,fiat,128,3,5,...,0,0,0,0,0,0,0,0,0,3
4,70,1,71,80,8,ford escort,ford,escort,2,4,...,0,0,0,0,0,0,0,0,0,4
5,70,1,134,159,16,mercedes 200-300,mercedes,200,2,4,...,0,0,0,0,0,0,0,0,0,5
6,70,1,165,197,19,opel kadett,opel,kadet,2,4,...,0,0,0,0,0,0,0,0,0,6
7,70,1,172,194,19,opel rekord,opel,record,2,4,...,0,0,0,0,0,0,0,0,0,7
8,70,1,186,202,20,peugeot 504,peugeot,504,1,3,...,0,0,0,0,0,0,0,0,0,8
9,70,1,187,207,20,peugeot 304,peugeot,304,1,3,...,0,0,0,0,0,0,0,0,0,9


In [542]:
test['y'].nunique()

40

In [543]:
test['s'].sum()

0.9999999999999979

$$
u_{i j h}=\mathbf{x}_{i j} \boldsymbol{\beta}_o+\varepsilon_{i j h}, \quad j=1, \ldots, J
$$

where: 

- $i$ is the $\textit{country-year}$ pair
- $j$ is the alternative car
- $h$ is the household

First off: Are we: 

    1. interested in the marginal utility of a car's characteristic (conditional logit) or 
    
    2. the change in utility of car $j$ relative to car 1 given a change in household characteristics?

In this assignment, we are examning home bias - that is the propensity to choose a car manufactured in the home country. We are therefore interested in 1) and will use a conditional logit model.

In [544]:
x.shape

(150, 40, 9)

In [545]:
# The starting values to be passed to our optimizer
theta_start = clogit.starting_values(y, x)
theta_start.shape

(9,)

In [546]:
u = clogit.util(theta_start, x)
u.shape

(150, 40)

In [547]:
# Our conditional choice probabilities
# For coefficients ('theta') starting at zero, these must be equal to zero
# Intuition: No utility is gained by any car characteristics, thus market shares must be equal. Let's check this.
ccp = clogit.choice_prob(theta_start, x)
(ccp == 1/J) # all choice probs are equal to each other.

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [548]:
# In the first market-year, what are the choice-probs? (Given individuals place no weight on any car characteristics)
ccp.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [549]:
print(ccp.shape)
np.unique(ccp) #Why do we have NaN?

(150, 40)


array([0.025])

In [551]:
print(theta_start.shape, y.shape, x.shape)

(9,) (150, 40) (150, 40, 9)


In [552]:
np.linalg.matrix_rank(x) # something is not right when you include the car-brand dummies -> rank-condition is not fulfilled -> matrix becomes singular
# Reference category is 'BMW'
# Could be that none of the top 40 cars sold in a given year and market is home-made and/or that they are BMW's -> column(s) becomes zero.

array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7], dtype=int64)

In [553]:
pd.DataFrame(x[0])

,0,1,2,3,4,5,6,7,8
0,0.122399,0.0,1760.0,59.0,1050.0,8.900000,0.25,9660000.0,20363.7252
1,-0.794100,0.0,435.0,17.5,560.0,5.500000,0.25,9660000.0,20363.7252
2,-0.713977,0.0,435.0,19.0,590.0,5.500000,0.25,9660000.0,20363.7252
3,-0.417193,0.0,1116.0,40.5,785.0,8.000000,0.25,9660000.0,20363.7252
4,-0.491411,0.0,1098.0,29.5,825.0,8.200000,0.25,9660000.0,20363.7252
5,0.459705,0.0,1988.0,70.0,1330.0,10.900000,0.25,9660000.0,20363.7252
6,-0.461060,0.0,1078.0,33.0,745.0,8.766666,0.25,9660000.0,20363.7252
7,-0.165561,0.0,1492.0,29.5,1025.0,10.500000,0.25,9660000.0,20363.7252
8,0.143035,0.0,1796.0,56.0,1200.0,11.600000,0.25,9660000.0,20363.7252
9,-0.078272,0.0,1288.0,43.0,915.0,10.100000,0.25,9660000.0,20363.7252


In [554]:
test2 = pd.DataFrame(x[50])
test2.columns=x_vars
test2

,logp,home,cy,hp,we,li,tax,pop,inc
0,-0.438650,0.0,1300.0,63.0,940.0,7.8,0.25,56730000.0,29513.0262
1,-0.226922,0.0,1800.0,66.0,1050.0,7.6,0.25,56730000.0,29513.0262
2,-0.143563,0.0,1600.0,75.0,1080.0,8.5,0.25,56730000.0,29513.0262
3,0.407682,0.0,2000.0,110.0,1425.0,9.8,0.25,56730000.0,29513.0262
4,-0.891653,1.0,1000.0,33.0,640.0,5.0,0.25,56730000.0,29513.0262
5,-0.473815,1.0,1400.0,53.0,900.0,6.9,0.25,56730000.0,29513.0262
6,0.053274,1.0,2000.0,85.0,1280.0,8.3,0.25,56730000.0,29513.0262
7,-1.134503,0.0,750.0,25.0,700.0,5.6,0.25,56730000.0,29513.0262
8,-0.986177,0.0,900.0,33.0,740.0,6.2,0.25,56730000.0,29513.0262
9,-0.582270,0.0,1400.0,53.0,945.0,7.1,0.25,56730000.0,29513.0262


In [556]:
res = est.estimate(clogit.q, theta_start, y[:, 0], x)

Optimization terminated successfully.
         Current function value: 3.192185
         Iterations: 21
         Function evaluations: 270
         Gradient evaluations: 27


In [557]:
res['theta']

array([ 6.42855096e+00,  1.21097154e-01,  2.04601799e-04,  8.99830539e-03,
       -8.71445571e-03, -3.25885047e-01, -1.20239951e-06, -1.85070248e-07,
        0.00000000e+00])

In [558]:
theta_res = res['theta']

ccp_res = clogit.choice_prob(theta_res, x)

In [559]:
ccp_res

array([[0.03042623, 0.00955542, 0.01248148, ..., 0.00516778, 0.00748987,
        0.00374895],
       [0.0280647 , 0.04121252, 0.03539991, ..., 0.02544964, 0.01216756,
        0.00995351],
       [0.07507805, 0.02401308, 0.02165588, ..., 0.01201194, 0.02049831,
        0.00643405],
       ...,
       [0.01987101, 0.0575612 , 0.03038645, ..., 0.02091369, 0.00750684,
        0.00701568],
       [0.02411644, 0.06776901, 0.01009842, ..., 0.00275334, 0.00756803,
        0.01486795],
       [0.0339414 , 0.09698301, 0.00933931, ..., 0.01292417, 0.02049899,
        0.01357216]])

In [560]:
ccp.shape

(150, 40)

In [588]:
market_share = cars['s'].values.reshape((N,J))
market_share

array([[0.01129646, 0.01464355, 0.02803195, ..., 0.02091936, 0.03233297,
        0.02175614],
       [0.01889573, 0.01742654, 0.02658294, ..., 0.01223223, 0.03081517,
        0.02623697],
       [0.00814136, 0.02169082, 0.01692497, ..., 0.0149763 , 0.01093299,
        0.02895637],
       ...,
       [0.02337781, 0.00867737, 0.00884787, ..., 0.01159728, 0.01712509,
        0.00843496],
       [0.0212737 , 0.01036163, 0.00758375, ..., 0.00859487, 0.02086731,
        0.01445988],
       [0.02890046, 0.00973071, 0.02472666, ..., 0.03594627, 0.01344469,
        0.01046751]])

In [589]:
market_share.shape

(150, 40)

In [591]:
ccp_res.shape

(150, 40)

In [604]:
log_lik_cont = market_share.T @ np.log(ccp)

In [605]:
log_lik_cont

array([[-10.85122272, -10.85122272, -10.85122272, ..., -10.85122272,
        -10.85122272, -10.85122272],
       [-11.04756788, -11.04756788, -11.04756788, ..., -11.04756788,
        -11.04756788, -11.04756788],
       [-12.12996443, -12.12996443, -12.12996443, ..., -12.12996443,
        -12.12996443, -12.12996443],
       ...,
       [-15.0850272 , -15.0850272 , -15.0850272 , ..., -15.0850272 ,
        -15.0850272 , -15.0850272 ],
       [ -9.81304041,  -9.81304041,  -9.81304041, ...,  -9.81304041,
         -9.81304041,  -9.81304041],
       [ -9.02836995,  -9.02836995,  -9.02836995, ...,  -9.02836995,
         -9.02836995,  -9.02836995]])

In [606]:
beta_hat = np.argmin(-1/N*log_lik_cont, axis=1)

In [607]:
beta_hat.shape

(40,)

In [608]:
beta_hat

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)